In [ ]:
class CFG:
    exp = '005'

# ====================================================
# Colab settings
# ====================================================
class ColabConfig:
    dataset_name = 'PPPM-analyze'
    dataset_version = 'exp' + CFG.exp
    dataset_new = True  # 新しいデータセットか
    dataset_dir = None  # Kaggle Dataset にアップロードするディレクトリ
    dataset_note = '""'  # 前の版からの変更点
    in_colab = False  # colab上にデータダウンロード

In [ ]:
!nvcc --version
!python -c 'import torch; print(torch.__version__) '
!python --version
print('')
!nvidia-smi
print('')
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0
1.11.0+cu113
Python 3.7.13

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.


Your runtime has 13.6 gigabytes of available RAM



In [ ]:
%%time
import sys
COLAB = "google.colab" in sys.modules

if COLAB:
    import os
    print('This environment is Google Colab')

    # mount drive
    from google.colab import drive
    if not os.path.isdir('/content/drive'):
        drive.mount('/content/drive')
    os.chdir('/content/drive/MyDrive/kaggle/PPPM/exps')

    # kaggle api token and update kaggle api
    from google.colab import files
    if not os.path.isfile('~/.kaggle/kaggle.json'):
        # files.upload()
        !mkdir -p ~/.kaggle
        !cp /content/drive/MyDrive/kaggle/kaggle.json ~/.kaggle/
        !chmod 600 ~/.kaggle/kaggle.json
        !pip install --upgrade --force-reinstall --no-deps -q kaggle

    if ColabConfig.in_colab:
        # make directory in colab
        !mkdir -p /content/input
        !mkdir -p /content/working

        # download dataset in colab
        import zipfile, glob
        os.chdir('/content/input')
        # !kaggle competitions download -qc birdclef-2022 -p birdclef-2022
        
        # !mkdir birdclef-2022
        # !cp /content/drive/MyDrive/kaggle/BirdCLEF2022/input/birdclef-2022/* ./birdclef-2022/

        for p in glob.glob('**/*.zip', recursive=True):
            print(p)
            d, f = os.path.split(p)
            # if f in ['']:
            #     continue
            with zipfile.ZipFile(p, 'r') as zipf:
                print('unzip: ', zipf)
                zipf.extractall(d)
                print('remove: ', f)
                os.remove(p)
        
        os.chdir('/content/working')

This environment is Google Colab
Mounted at /content/drive
     |████████████████████████████████| 58 kB 4.0 MB/s 
CPU times: user 1.38 s, sys: 312 ms, total: 1.7 s
Wall time: 20.6 s


In [ ]:
import os
import pickle
from pathlib import Path

import numpy as np
import pandas as pd
pd.set_option('display.width', 10000)
pd.set_option('display.max_colwidth', None)
import torch
import lightgbm as lgb

In [ ]:
oof_dir = Path(f'/content/drive/MyDrive/kaggle/PPPM/output/{CFG.exp}')

In [ ]:
oof_df = pd.read_pickle(oof_dir / 'oof_df.pkl')
oof_df.head()

,id,anchor,target,context,score,context_text,text,text_rev,fold,pred
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50,HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLE...,abatement. [SEP] abatement of pollution. [SEP]...,abatement of pollution. [SEP] abatement. [SEP]...,0,0.411608
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75,HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLE...,abatement. [SEP] act of abating. [SEP] HUMAN N...,act of abating. [SEP] abatement. [SEP] HUMAN N...,0,0.595921
2,36d72442aefd8232,abatement,active catalyst,A47,0.25,HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLE...,abatement. [SEP] active catalyst. [SEP] HUMAN ...,active catalyst. [SEP] abatement. [SEP] HUMAN ...,0,0.168890
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50,HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLE...,abatement. [SEP] eliminating process. [SEP] HU...,eliminating process. [SEP] abatement. [SEP] HU...,0,0.438424
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00,HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLE...,abatement. [SEP] forest region. [SEP] HUMAN NE...,forest region. [SEP] abatement. [SEP] HUMAN NE...,0,0.000843


In [ ]:
y_pred = oof_df['pred'].values
y_true = oof_df['score'].values
oof_df['bce_loss'] = - y_true * np.log(y_pred + 1e-9) - (1-y_true) * np.log((1-y_pred) + 1e-9)
oof_df['mse_loss'] = np.square(y_pred - y_true)
print(oof_df['bce_loss'].mean())
print(oof_df['mse_loss'].mean())
oof_df.drop(['text', 'text_rev'], axis=1).head()

0.5535556574598449
0.02138652675587868


,id,anchor,target,context,score,context_text,fold,pred,bce_loss,mse_loss
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50,HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL,0,0.411608,0.709023,7.813123e-03
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75,HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL,0,0.595921,0.614772,2.374040e-02
2,36d72442aefd8232,abatement,active catalyst,A47,0.25,HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL,0,0.168890,0.583372,6.578832e-03
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50,HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL,0,0.438424,0.700789,3.791645e-03
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00,HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL,0,0.000843,0.000843,7.108307e-07


In [ ]:
oof_df.sort_values('mse_loss', ascending=False).drop(['text', 'text_rev'], axis=1).head(10)

,id,anchor,target,context,score,context_text,fold,pred,bce_loss,mse_loss
10802,3c8e8b15d2721b27,channel vectors,channel of vectors,H04,0.00,ELECTRICITY. ELECTRIC COMMUNICATION TECHNIQUE,1,0.975567,3.711830,0.951731
10807,5d9e69bd63e69979,channel vectors,channels of vectors,H04,0.00,ELECTRICITY. ELECTRIC COMMUNICATION TECHNIQUE,1,0.973451,3.628768,0.947607
29190,5ad513a964e9d0a6,dissolving,dissolution,C22,0.00,HEMISTRY. METALLURGY. METALLURGY. FERROUS OR NON-FERROUS ALLOYS. TREATMENT OF ALLOYS OR NON-FERROUS METALS,3,0.937965,2.780061,0.879779
15574,6af4f2d9506545f2,punch face,face punch,B05,0.00,"PERFORMING OPERATIONS. TRANSPORTING. SPRAYING OR ATOMISING IN GENERAL. APPLYING FLUENT MATERIALS TO SURFACES, IN GENERAL",1,0.868551,2.029136,0.754381
14108,aba48e6f94149e6f,multiplexed data,multiplexed date,H01,0.00,ELECTRICITY. BASIC ELECTRIC ELEMENTS,1,0.859290,1.961055,0.738379
17076,80efbcfb3bc66967,sphygmomanometer,blood pressure,G09,1.00,PHYSICS. EDUCATION. CRYPTOGRAPHY. DISPLAY. ADVERTISING. SEALS,1,0.159857,1.833475,0.705840
19193,f297ea79eab86071,boom hydraulic cylinder,boom a hydraulic cylinder,E02,0.00,FIXED CONSTRUCTIONS. HYDRAULIC ENGINEERING. FOUNDATIONS. SOIL SHIFTING,2,0.816851,1.697454,0.667245
1475,e76a9afbecc76ef7,catching surface,catching nose,F03,0.75,"MECHANICAL ENGINEERING. LIGHTING. HEATING. WEAPONS. BLASTING. MACHINES OR ENGINES FOR LIQUIDS. WIND, SPRING, OR WEIGHT MOTORS. PRODUCING MECHANICAL POWER OR A REACTIVE PROPULSIVE THRUST, NOT OTHERWISE PROVIDED FOR",0,0.000152,6.595730,0.562273
30216,336423dca70783c2,gnd,earth terminal,G01,0.75,PHYSICS. MEASURING. TESTING,3,0.000207,6.363421,0.562190
13609,4dc649edbba5536b,lifting load,weight lifting,B66,0.75,PERFORMING OPERATIONS. TRANSPORTING. HOISTING. LIFTING. HAULING,1,0.000324,6.025214,0.562013


In [ ]:
# 不適切な範囲の予測はなさそう
oof_df[(oof_df['pred'] < 0) & (oof_df['pred'] > 1)]

,id,anchor,target,context,score,context_text,text,text_rev,fold,pred,bce_loss,mse_loss
